In [4]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/datamart.zenodo.3733554.87ed9e35-0f51-49cb-a73c-c0e3581051f5')

In [5]:
data=resp.content.decode()
list=[x.split(',') for x in data.split('\n')]

df=pd.DataFrame(list[1:],columns=list[0])


In [6]:
id="datamart.zenodo.3733554.87ed9e35-0f51-49cb-a73c-c0e3581051f5"
filename=id.replace(".","_").replace("-","_")+".parquet"


In [15]:
df.to_parquet(filename)
df.to_csv('test_file.csv')

In [12]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    print(c['name'],c['structural_type'])

/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

Country http://schema.org/Text
Total cases http://schema.org/Integer
New cases http://schema.org/Text
Total deaths http://schema.org/Text
New deaths http://schema.org/Text
Total recovered http://schema.org/Text
Active cases http://schema.org/Integer
Critical cases http://schema.org/Text
Cases/1M pop http://schema.org/Float
Lon http://schema.org/Float
 http://schema.org/Text


In [14]:
con=duckdb.connect("testdb.db")
res=con.sql(" describe select * from "+filename).fetchdf()
con.sql("select * from "+filename+" where 'Total cases'='81054.0'")

┌─────────┬─────────────┬───────────┬──────────────┬───┬────────────────┬──────────────┬─────────┬─────────┐
│ Country │ Total cases │ New cases │ Total deaths │ … │ Critical cases │ Cases/1M pop │   Lon   │   Lat
   │
│ varchar │   varchar   │  varchar  │   varchar    │   │    varchar     │   varchar    │ varchar │ varchar │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                  0 rows                                                  │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [13]:
df['Total cases'].astype(int)

ValueError: invalid literal for int() with base 10: '81054.0'